In [1]:
print("test")

test

MY_C モジュール：全ての変数の定義、関数の定義を入れるところ

In [4]:
module MY_C

  #examples:

  #cannot change
  struct testObject
    NAME_this
  end

  #宣言の方法
  function testObject(_NAME_this)
    NAME_this = _NAME_this
    return testObject(NAME_this)
  end

  #can change
  mutable struct testObject_mut
    NAME_this
  end

  function testObject_mut(_NAME_this)
    NAME_this = _NAME_this
    return testObject_mut(NAME_this)
  end

  function myPrint(m)
    println(m)
  end


  #here newers:


  #agent class
  struct agentData
    NAME
    angle::Float64
    self_w::Float64
    other_w::Float64
    strategy
  end

  function agentData(_NAME,_angle,_strategy)
    NAME = _NAME
    angle = _angle
    strategy = _strategy
    return agentData(NAME,angle,cos(pi / 180 * angle),sin(pi / 180 * angle),strategy)
  end


end

Main.MY_C

In [6]:
using .MY_C

MY_C.myPrint("will")


will


# 相手の行動は固定、よってリストで与えることに。エージェントの出力形式に合わせればよいか？

やりたいことは、

・Juliaで

・相手のエージェントの行動はC（A、協力）固定の時に

・囚人のジレンマゲームで、参加者がAとBをランダムに出力するとして、

・10回、ゲームを行った場合に、

・参加者の出力するランダム配列を全通り試し、相手のSVOの予測平均を出す。

# 用意すべきもの

上記をするためには、

・マトリックスの用意（エージェント固定なので、Aの2点、Bの3点が必要かな。あとは、エージェントの獲得点数。）

・マトリックスからエージェントが出力する表情の計算

・記録するための10列のリスト、および全パターン実行の保存先

・模擬プレイヤーとして、AとBを出力する人。

・ベイズ更新の関数、保存式

以下は、最低限の条件

・エージェントの計算において、判定は下2桁でroundしたものを使う。

・エージェントの出力する表情は、sad、neutral、joyの3種類である（数値化する方が良い）

・実行された角度は、（今のところ）180.0、157.5、135.0、112.5～22.5、0.0、の5つであり、加えてneutral-neutralである。

# ページワン

・実行する角度リスト

・ジレンマゲームのマトリックス

・表情出力関数

In [5]:
#cos(pi / 180 * angle)

#実行する角度のリスト
angleList = [180.0, 157.5, 135.0, 90.0, 0.0]

#囚人のジレンマゲームのマトリックス
dillemmaMatrix = [2,0,3,1]

println(angleList[1])
println(dillemmaMatrixList[1])


#SVOから、表情を出力するfunction
function EmotionExpressionfromSVO(selfWeight,otherWeight,selfValue,otherValue)

    #重みから効用を計算。下2桁に。digitsで下何桁
    utility = round(selfWeight * selfValue + otherWeight * otherValue,digits=2)
    #println("utility=",utility)

    #判定のための変数
    # 0 : bad
    # 1 : neutral
    # 2 : joy　としたよ。
    emotion = 1

    if utility > 0.0
        emotion = 2
    elseif utility < 0.0
        emotion = 0
    end
    #println("emotion=",emotion)

    return emotion
end

EmotionExpressionfromSVO(1,0,2,2)


utility=2.0
emotion=2


2

Juliaのインテリセンスが効かないときは、不便かもだが、環境が遅い可能性あり。拡張機能を確認し、いろいろと再起動すべし。

# <span style="font-size: 50%; color: black;"> 角度を受け取り、SVOの重みに変換するfunction </span>

In [12]:
#角度を受け取り、重み2つを出力するfunction
function WeightfromSVO(angleSVO)
    return cos(pi / 180 * angleSVO),sin(pi / 180 * angleSVO)
end

print(WeightfromSVO(180))

(-1.0, 1.2246467991473532e-16)

# <span style="font-size: 50%; color: black;"> ランダムにA(1)かB(2)かを出力する、プレイヤーfunction </span>

In [14]:
#1か2をランダムに出力するプレイヤーfunction
function RandomofPlayer()
    rate = rand(1:2)
    #print(rate)
    return rate
end

RandomofPlayer()

1

# ゲームを10回分実行するfunction

・片方のエージェントは全て固定のA（1）しか選ばない。

・プレイヤーはランダムに選択していく。

In [17]:
#ゲームを10回行うfunction
#ゲームマトリックスと、エージェントのSVOを設定する必要がある。

function PlayGame(gameMatrix,agentsSVOAngle,round_n=10)
    #agentのSVO重みを計算
    agentsSelfWeight, agentsOtherWeight = WeightfromSVO(agentsSVOAngle)
    println("self=", agentsSelfWeight, " : other=", agentsOtherWeight)

    #指定のラウンド数まで実行する
    for i in 1:round_n
        #ラウンド数
        println("round ",i)

        

    end
end

PlayGame(dillemmaMatrix,180.0)

self=-1.0 : other=1.2246467991473532e-16
round 1
round 2
round 3
round 4
round 5
round 6
round 7
round 8
round 9
round 10


In [1]:
#使用前に
#juliaのパッケージモード（最初の画面で、]を入力すると、pkgと出る。終了はdelキー）にて
#add Statistics

Using Statistics

LoadError: syntax: extra token "Statistics" after end of expression